In [1]:
import sagemaker
import boto3
from sagemaker.amazon.amazon_estimator import get_image_uri 
from sagemaker.session import s3_input, Session

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/base_serializers.py:28: UserWarning: A NumPy version >=1.23.5 and <2.3.0 is required for this version of SciPy (detected version 1.22.4)
  import scipy.sparse


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [3]:
bucket_name = 'bankapplication420420420' # <--- CHANGE THIS VARIABLE TO A UNIQUE NAME FOR YOUR BUCKET
my_region = boto3.session.Session().region_name # set the region of the instance
print(my_region)

ap-south-1


In [6]:
s3 = boto3.resource('s3')
try:
    if  my_region == 'us-east-1':
        s3.create_bucket(Bucket=bucket_name)
    print('S3 bucket created successfully')
except Exception as e:
    print('S3 error: ',e)

S3 bucket created successfully


In [11]:
import boto3

bucket_name = 'bankapplication420420420'  # <--- Ensure this is a unique name for your bucket
my_region = boto3.session.Session().region_name  # Get the region of the current session
print(f"Current region: {my_region}")

s3 = boto3.resource('s3')

try:
    if my_region == 'us-east-1':
        # 'us-east-1' does not require specifying a location constraint
        s3.create_bucket(Bucket=bucket_name)
    else:
        # For other regions, you need to specify the region
        s3.create_bucket(
            Bucket=bucket_name,
            CreateBucketConfiguration={
                'LocationConstraint': my_region
            }
        )
    print('S3 bucket created successfully')
except Exception as e:
    print('S3 error: ', e)


Current region: ap-south-1
S3 bucket created successfully


In [12]:
prefix = 'xgboost-as-a-built-in-algo'
output_path ='s3://{}/{}/output'.format(bucket_name, prefix)
print(output_path)

s3://bankapplication420420420/xgboost-as-a-built-in-algo/output


In [13]:
import pandas as pd
import urllib
try:
    urllib.request.urlretrieve ("https://d1.awsstatic.com/tmt/build-train-deploy-machine-learning-model-sagemaker/bank_clean.27f01fbbdf43271788427f3682996ae29ceca05d.csv", "bank_clean.csv")
    print('Success: downloaded bank_clean.csv.')
except Exception as e:
    print('Data load error: ',e)

try:
    model_data = pd.read_csv('./bank_clean.csv',index_col=0)
    print('Success: Data loaded into dataframe.')
except Exception as e:
    print('Data load error: ',e)

Success: downloaded bank_clean.csv.
Success: Data loaded into dataframe.


In [14]:
import numpy as np
train_data, test_data = np.split(model_data.sample(frac=1, random_state=1729), [int(0.7 * len(model_data))])
print(train_data.shape, test_data.shape)

(28831, 61) (12357, 61)


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/numpy/core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [15]:
import os
pd.concat([train_data['y_yes'], train_data.drop(['y_no', 'y_yes'], 
                                                axis=1)], 
                                                axis=1).to_csv('train.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
s3_input_train = sagemaker.s3_input(s3_data='s3://{}/{}/train'.format(bucket_name, prefix), content_type='csv')
     

AttributeError: module 'sagemaker' has no attribute 's3_input'

In [17]:
import os
import boto3
import sagemaker
from sagemaker.inputs import TrainingInput

# Combine the columns and save to a CSV file
pd.concat([train_data['y_yes'], train_data.drop(['y_no', 'y_yes'], axis=1)], axis=1).to_csv('train.csv', index=False, header=False)

# Upload the CSV file to S3
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')

# Create a TrainingInput object for the training data
s3_input_train = TrainingInput(s3_data='s3://{}/{}/train'.format(bucket_name, prefix), content_type='csv')


In [18]:
# Combine the columns and save to a CSV file for test data
pd.concat([test_data['y_yes'], test_data.drop(['y_no', 'y_yes'], axis=1)], axis=1).to_csv('test.csv', index=False, header=False)

# Upload the test CSV file to S3
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'test/test.csv')).upload_file('test.csv')

# Create a TrainingInput object for the test data
s3_input_test = TrainingInput(s3_data='s3://{}/{}/test'.format(bucket_name, prefix), content_type='csv')

In [19]:
container = get_image_uri(boto3.Session().region_name,
                          'xgboost', 
                          repo_version='1.0-1')

The method get_image_uri has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [20]:
# Use sagemaker.image_uris.retrieve instead of get_image_uri
container = sagemaker.image_uris.retrieve(
    framework='xgboost',
    region=boto3.Session().region_name,
    version='1.0-1'
)

In [21]:
hyperparameters = {
        "max_depth":"5",
        "eta":"0.2",
        "gamma":"4",
        "min_child_weight":"6",
        "subsample":"0.7",
        "objective":"binary:logistic",
        "num_round":50
        }

In [23]:
estimator = sagemaker.estimator.Estimator(image_uri=container, 
                                          hyperparameters=hyperparameters,
                                          role=sagemaker.get_execution_role(),
                                          train_instance_count=1, 
                                          train_instance_type='ml.m5.2xlarge', 
                                          train_volume_size=5, # 5 GB 
                                          output_path=output_path,
                                          train_use_spot_instances=True,
                                          train_max_run=300,
                                          train_max_wait=600)

train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_max_run has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_use_spot_instances has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_max_wait has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_volume_size has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [24]:
estimator.fit({'train': s3_input_train,'validation': s3_input_test})

INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2024-08-31-04-00-06-585


2024-08-31 04:00:06 Starting - Starting the training job...
2024-08-31 04:00:27 Starting - Preparing the instances for training...
2024-08-31 04:01:06 Downloading - Downloading the training image...
2024-08-31 04:01:42 Training - Training image download completed. Training in progress....
2024-08-31 04:02:07 Uploading - Uploading generated training model[2024-08-31 04:01:58.715 ip-10-0-162-104.ap-south-1.compute.internal:7 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','


In [25]:
xgb_predictor = estimator.deploy(initial_instance_count=1,instance_type='ml.m4.xlarge')

INFO:sagemaker:Creating model with name: sagemaker-xgboost-2024-08-31-04-06-04-801
INFO:sagemaker:Creating endpoint-config with name sagemaker-xgboost-2024-08-31-04-06-04-801
INFO:sagemaker:Creating endpoint with name sagemaker-xgboost-2024-08-31-04-06-04-801


-----!

In [26]:
from sagemaker.predictor import csv_serializer
test_data_array = test_data.drop(['y_no', 'y_yes'], axis=1).values #load the data into an array
xgb_predictor.content_type = 'text/csv' # set the data type for an inference
xgb_predictor.serializer = csv_serializer # set the serializer type
predictions = xgb_predictor.predict(test_data_array).decode('utf-8') # predict!
predictions_array = np.fromstring(predictions[1:], sep=',') # and turn the prediction into an array
print(predictions_array.shape)

ImportError: cannot import name 'csv_serializer' from 'sagemaker.predictor' (/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/predictor.py)

In [27]:
from sagemaker.predictor import csv_serializer
import numpy as np

# Load the data into an array
test_data_array = test_data.drop(['y_no', 'y_yes'], axis=1).values

# Set the content type and serializer
xgb_predictor.content_type = 'text/csv'
xgb_predictor.serializer = csv_serializer

# Get predictions
predictions = xgb_predictor.predict(test_data_array)

# Check if predictions need decoding
if isinstance(predictions, bytes):
    predictions = predictions.decode('utf-8')

# Convert predictions to an array
predictions_array = np.fromstring(predictions, sep=',')

# Print the shape of the predictions array
print(predictions_array.shape)


ImportError: cannot import name 'csv_serializer' from 'sagemaker.predictor' (/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/predictor.py)

In [28]:
from sagemaker.serializers import CSVSerializer
import numpy as np

# Load the data into an array
test_data_array = test_data.drop(['y_no', 'y_yes'], axis=1).values

# Set the content type and serializer
xgb_predictor.content_type = 'text/csv'
xgb_predictor.serializer = CSVSerializer()

# Get predictions
predictions = xgb_predictor.predict(test_data_array)

# Check if predictions need decoding
if isinstance(predictions, bytes):
    predictions = predictions.decode('utf-8')

# Convert predictions to an array
predictions_array = np.fromstring(predictions, sep=',')

# Print the shape of the predictions array
print(predictions_array.shape)


(12357,)


In [29]:
sagemaker.Session().delete_endpoint(xgb_predictor.endpoint)
bucket_to_delete = boto3.resource('s3').Bucket(bucket_name)
bucket_to_delete.objects.all().delete()
     

See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
INFO:sagemaker:Deleting endpoint with name: sagemaker-xgboost-2024-08-31-04-06-04-801


[{'ResponseMetadata': {'RequestId': 'TD3AMSWEKZDFDSPH',
   'HostId': 'wBlpkmlZGt125soPcHBDO+jsU7qXxUQEdolMi9KrRsj4vg1pymXHdEVdJcMIkQH+EQmZqED2XRY=',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amz-id-2': 'wBlpkmlZGt125soPcHBDO+jsU7qXxUQEdolMi9KrRsj4vg1pymXHdEVdJcMIkQH+EQmZqED2XRY=',
    'x-amz-request-id': 'TD3AMSWEKZDFDSPH',
    'date': 'Sat, 31 Aug 2024 04:21:14 GMT',
    'content-type': 'application/xml',
    'transfer-encoding': 'chunked',
    'server': 'AmazonS3',
    'connection': 'close'},
   'RetryAttempts': 0},
  'Deleted': [{'Key': 'xgboost-as-a-built-in-algo/test/test.csv'},
   {'Key': 'xgboost-as-a-built-in-algo/output/sagemaker-xgboost-2024-08-31-04-00-06-585/debug-output/index/000000000/000000000000_worker_0.json'},
   {'Key': 'xgboost-as-a-built-in-algo/output/sagemaker-xgboost-2024-08-31-04-00-06-585/debug-output/index/000000000/000000000040_worker_0.json'},
   {'Key': 'xgboost-as-a-built-in-algo/output/sagemaker-xgboost-2024-08-31-04-00-06-585/output/model.tar.gz'}